<a href="https://colab.research.google.com/github/jin1244411211212/SmartThingsPublic/blob/master/test%20%EC%97%94%EB%B2%A0%EC%9D%B4%EB%94%A9.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

!pip install sentence-transformers pandas numpy scikit-learn




Mounted at /content/drive
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 91.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 59.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 46.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 11.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 9.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 60.2 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully u

In [2]:
import pandas as pd
import numpy as np
from sentence_transformers import SentenceTransformer
import os
import pickle

# --- 데이터베이스 파일 경로 설정 ---
db_path = '/content/drive/MyDrive/Colab Notebooks/venture_global_db.pkl'

# --- 단계 1: 데이터 준비 및 청킹 ---
# 제공된 전체 텍스트를 구조화합니다.
knowledge_chunks = [
    # 피라미드 꼭대기 (Apex): 핵심 투자 가설
    {'text': "Venture Global의 성장 동력은 AI의 기술적 필요성(안정적 기저부하 전력)과 미국의 패권 유지를 위한 정치적 필요성(무역적자 해소)이라는 두 개의 강력한 엔진이 동시에 작동하고 있습니다. 이는 수요 기반이 예상보다 훨씬 더 견고하고 다층적일 수 있음을 의미합니다.", 'metadata': {'pyramid_level': '꼭대기', 'topic': '핵심 투자 아이디어', 'source': '분석 결론'}},
    {'text': "현재의 차트는 성장 잠재력(CP2 승인)과 소송 리스크가 충돌하며 만들어낸 '힘의 균형점'을 시각적으로 보여주는 교과서와 같습니다. 내부자 매수 가격($9~10)은 심리적 최저 방어선을, 최근 돌파 후 지지선($16.50)은 기술적 1차 방어선을 제공합니다.", 'metadata': {'pyramid_level': '꼭대기', 'topic': '기술적 분석 결론', 'source': '분석 결론'}},
    {'text': "Venture Global의 재무 리스크를 평가할 때, 우리는 '부채 총액'이 아니라 예측 불가능한 리스크인 '소송 결과'에 집중해야 합니다. 매 분기 발표되는 '영업활동 현금흐름'이 이 리스크를 감당할 체력이 되는지 확인하는 것이 가장 중요합니다.", 'metadata': {'pyramid_level': '꼭대기', 'topic': '재무 분석 결론', 'source': '분석 결론'}},
    {'text': "Venture Global의 비즈니스 모델은 단순한 LNG 판매업이 아니라, '빠르게 인프라를 건설하여 시장의 기회를 선점하는 일종의 부동산 개발업'과 유사한 특성을 가집니다. 미래 파이프라인의 가치를 평가하는 관점이 필요하며, 실행 리스크를 관리하는 것이 투자 성공의 열쇠가 될 것입니다.", 'metadata': {'pyramid_level': '꼭대기', 'topic': '비즈니스 모델 결론', 'source': '분석 결론'}},

    # 피라미드 중간층 (Context): 주요 분석 및 해설
    {'text': "빌 게이츠의 '테라파워'(SMR)나 ESS 기술 발전은 '전력 수요 증가 = 천연가스 수요 증가' 공식에 대한 잠재적 리스크입니다. 기술 발전 변수를 항상 염두에 두어야 합니다.", 'metadata': {'pyramid_level': '중간층', 'topic': '리스크 분석 (대체에너지)', 'source': '해설 및 비판적 사고'}},
    {'text': "카타르와 중국의 초장기 계약처럼, 각 국가는 에너지 안보를 위해 공급처를 다변화하려 하므로 미국의 정치적 압박이 항상 성공할 것이라는 가정은 지나치게 낙관적일 수 있습니다.", 'metadata': {'pyramid_level': '중간층', 'topic': '리스크 분석 (지정학)', 'source': '해설 및 비판적 사고'}},
    {'text': "회사의 막대한 부채는 대부분 개별 프로젝트의 미래 수익성을 담보로 한 '프로젝트 파이낸싱(PF)'입니다. 이는 리스크가 분리된 구조로, 부채 총액 숫자만 보고 과도하게 우려할 필요는 없습니다.", 'metadata': {'pyramid_level': '중간층', 'topic': '재무 분석 (부채 성격)', 'source': '해설 및 비판적 사고'}},
    {'text': "시운전 기간 중 현물 판매는 높은 수익을 안겨주지만, 가격 하락기에는 오히려 독이 될 수 있으며, 고객사와의 소송의 핵심 원인이기도 한 '양날의 검'입니다.", 'metadata': {'pyramid_level': '중간층', 'topic': '비즈니스 모델 분석 (시운전 수익)', 'source': '해설 및 비판적 사고'}},
    {'text': "가치평가 시 '124 MTPA'라는 미래의 꿈을 현재 가치로 착각해서는 안됩니다. 소송 리스크와 시간 가치에 대한 할인을 반드시 적용해야 합니다.", 'metadata': {'pyramid_level': '중간층', 'topic': '가치평가 분석', 'source': '해설 및 비판적 사고'}},


    # 피라미드 기반 (Base): 보고서 원문
    {'text': "데이터센터는 2030까지 CAGR 16% 추정(Goldman), 이전 추정(15%) 대비 증가한 수치", 'metadata': {'pyramid_level': '기반', 'topic': '에너지 수요', 'source': '보고서 원문'}},
    {'text': "미국의 무역 적자는 코로나 19를 기점으로 2배 이상 확대되었으며 이를 해결하기 위해 미국은 LNG 수입을 지속해서 압박하고 있음", 'metadata': {'pyramid_level': '기반', 'topic': '지정학', 'source': '보고서 원문'}},
    {'text': "추세선을 대략적으로 그어보면 하락 채널의 상단에 걸쳐있는 것을 확인할 수 있음... 채널 상단을 돌파할 경우 하락 추세선이 저항선에서 지지선으로 바뀔 수 있는 자리", 'metadata': {'pyramid_level': '기반', 'topic': '차트 분석', 'source': '보고서 원문'}},
    {'text': "CEO 마이클 사밸을 비롯한 다수 내부자가 3월 중순 경 9~10불에 지속적으로 매수한 바 있음... VG의 주가 하방을 지지하는 역할을 할 수 있을 것", 'metadata': {'pyramid_level': '기반', 'topic': '내부자 거래', 'source': '보고서 원문'}},
    {'text': "22년과 23년 매출과 영업이익이 높게 잡힌 것은 러-우 전쟁 당시 높은 현물 가격 속에서 상업적 운영(시운전)이 길어짐에 따라 막대한 이익을 벌어들였기 때문임", 'metadata': {'pyramid_level': '기반', 'topic': '손익계산서 (시운전 수익)', 'source': '보고서 원문'}},
    {'text': "총 장기차입금은 29.3B이며 부채는 28년부터 상환 기간이 본격적으로 도래함", 'metadata': {'pyramid_level': '기반', 'topic': '재무상태표 (부채)', 'source': '보고서 원문'}},
    {'text': "벤처글로벌은 베이커휴즈 사에 액화 트레인을 주문 후 배로 수송을 받아 LNG 터미널 건설지에서 최종적으로 완성하는 형태로 사업을 운영하고 있음.", 'metadata': {'pyramid_level': '기반', 'topic': '모듈식 공정', 'source': '보고서 원문'}},
    {'text': "벤처글로벌은 MTPA 기준 계약 물량이 26년 22에서 30년 124까지 늘어날 수 있는 Upside를 가지고 있음", 'metadata': {'pyramid_level': '기반', 'topic': '미래 파이프라인', 'source': '보고서 원문'}},
    {'text': "Edison, BP, Shell 등 업체가 Venture Global이 Calcasieu Pass LNG 공급 계약을 위반했다고 주장. 지불해야 할 금액이 상당할 수 있음", 'metadata': {'pyramid_level': '기반', 'topic': '계약 분쟁 소송', 'source': '보고서 원문'}},
    {'text': "셰니어 시총 / 용량 = 43B/65Mtpa = 0.66B/Mtpa ... 벤처글로벌 용량 X 용량당 시총 = 124Mtpa X 0.66 = 81.84B(주당 33.85$)", 'metadata': {'pyramid_level': '기반', 'topic': '상대 가치평가', 'source': '보고서 원문'}},
    {'text': "트럼프가 Venture Global의 CEO를 비롯한 에너지 임원들에게 선거운동을 위해 10억 달러 모금을 요청했으며, 신규 LNG 수출 허가 동결 해제 등을 약속했다고 알려짐.", 'metadata': {'pyramid_level': '기반', 'topic': '정치적 수혜 (트럼프)', 'source': '보고서 원문'}},
]

print(f"총 {len(knowledge_chunks)}개의 지식 청크를 준비했습니다.")

# --- 단계 2: 벡터화 실행 및 데이터베이스 구축 ---
print("\n임베딩 모델을 로드합니다... (처음 실행 시 시간이 걸릴 수 있습니다)")
model = SentenceTransformer('jhgan/ko-sroberta-multitask')

print("문서 벡터화를 시작합니다...")
corpus_texts = [chunk['text'] for chunk in knowledge_chunks]
corpus_embeddings = model.encode(corpus_texts, convert_to_tensor=False, show_progress_bar=True) # convert_to_tensor=False로 변경

# 데이터프레임 생성
df = pd.DataFrame(knowledge_chunks)
df['embedding'] = list(corpus_embeddings)

# --- 단계 3: Google Drive에 데이터베이스 저장 ---
with open(db_path, 'wb') as f:
    pickle.dump(df, f)

print(f"\n벡터 데이터베이스 구축 완료! '{db_path}'에 저장되었습니다.")
print("데이터베이스 구조 (상위 5개):")
print(df.head())

총 20개의 지식 청크를 준비했습니다.

임베딩 모델을 로드합니다... (처음 실행 시 시간이 걸릴 수 있습니다)


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/123 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/4.86k [00:00<?, ?B/s]

...

special_tokens_map.json:   0%|          | 0.00/156 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

문서 벡터화를 시작합니다...


Batches:   0%|          | 0/1 [00:00<?, ?it/s]


벡터 데이터베이스 구축 완료! '/content/drive/MyDrive/Colab Notebooks/venture_global_db.pkl'에 저장되었습니다.
데이터베이스 구조 (상위 5개):
                                                text  \
0  Venture Global의 성장 동력은 AI의 기술적 필요성(안정적 기저부하 전력...   
1  현재의 차트는 성장 잠재력(CP2 승인)과 소송 리스크가 충돌하며 만들어낸 '힘의 ...   
2  Venture Global의 재무 리스크를 평가할 때, 우리는 '부채 총액'이 아니...   
3  Venture Global의 비즈니스 모델은 단순한 LNG 판매업이 아니라, '빠르...   
4  빌 게이츠의 '테라파워'(SMR)나 ESS 기술 발전은 '전력 수요 증가 = 천연가...   

                                            metadata  \
0  {'pyramid_level': '꼭대기', 'topic': '핵심 투자 아이디어'...   
1  {'pyramid_level': '꼭대기', 'topic': '기술적 분석 결론',...   
2  {'pyramid_level': '꼭대기', 'topic': '재무 분석 결론', ...   
3  {'pyramid_level': '꼭대기', 'topic': '비즈니스 모델 결론'...   
4  {'pyramid_level': '중간층', 'topic': '리스크 분석 (대체에...   

                                           embedding  
0  [-0.30434853, 0.016520511, -0.22050075, -0.478...  
1  [0.009305086, 0.11005468, 0.84904087, -0.60373...  
2  [-0.093938105, -0.07784459, -0.1650089, -0.663..